<a href="https://colab.research.google.com/github/sipocz/malavi/blob/B20210507/Malavi_LSTM_KAGGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
from keras.backend import clear_session

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
!rm Train.csv
fname="https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv"
!wget https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv

rm: cannot remove 'Train.csv': No such file or directory
--2021-05-07 17:13:04--  https://github.com/sipocz/malavi/raw/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/malavi/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv [following]
--2021-05-07 17:13:05--  https://raw.githubusercontent.com/sipocz/malavi/ccefa7562b0a115d207a02f2cf2754406870ef52/Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3158546 (3.0M) [text/plain]
Saving to: ‘Train.csv’

Train.csv           100%[===================>]   3.01M  --.-KB/s    in 0.03s   

20

In [3]:
df=pd.read_csv("Train.csv",sep=",")

In [4]:
c_df=[]
for i in range(len(df)):
    c_df.append(df.Text[i].replace(",","").replace(".","").replace("?","").replace("(","").replace(")","").replace("!","").replace(";","").replace("\n","").replace(":","").replace("<","").lower())


In [5]:
df["post"]=c_df

In [6]:
labels=['SOCIAL ISSUES', 'EDUCATION', 'RELATIONSHIPS', 'ECONOMY', 'RELIGION', 'POLITICS', 'LAW/ORDER', 'SOCIAL', 'HEALTH', 'ARTS AND CRAFTS', 'FARMING', 'CULTURE', 'FLOODING', 'WITCHCRAFT', 'MUSIC', 'TRANSPORT', 'WILDLIFE/ENVIRONMENT', 'LOCALCHIEFS', 'SPORTS', 'OPINION/ESSAY']

In [7]:
label_index=[]
for i in list(df.Label):
    label_index.append(labels.index(i))

In [8]:
df["label_index"]=label_index

In [9]:
df.post[1435]

' pac iunguza za boma la chifedulo nthumwi zomwe zimakumana mumzinda wa blantyre kukambirana za nkhani yakuti zigawo za dziko lino zizikhala ndi mtsogoleri wakewake pansi pa mtsogoleri wa dziko zati nkhaniyi kuti iyende bwino mpofunika kusintha malamulo ena mfundoyi ikugwirizana ndi zomwe adanena mphunzitsi wa za ndale kusukulu ya ukachenjede ya chancellor college blessings chinsinga kuti popanda kuunika malamulo nkhaniyi ikhoza kudzetsa chisokonezo  chisale watuluka nkumangidwanso  sipakala waimitsa nyumba ya malamulo  chakwera akwanitse malonjezohrdc choyamba tiunike kuti malamulo athu akutinji chifukwa mukhoza kukhala ndi zigawo zodziyimira pazokha zomwe zingamakolanenso chifukwa cha malamulo omwe mukutsata adatero chinsinga msonkhanowo udakonzedwa ndi bungwe la mipingo la public affairs committee pac ndipo cholinga chake chidali kuunika chomwe chidayambitsa nkhaniyi ndi kukambirana momwe ingayendere malinga ndi wapampando wa bungwe la pac mbusa felix chingota nthumwi za kumsonkhano

In [10]:
df.post[:].values

array([' mwangonde khansala wachinyamata akamati achinyamata ndi atsogoleri a mawa ambiri amaganiza kuti izi ndi nkhambakamwa chabe koma achinyamata ena monga lusubilo mwangonde akukwaniritsa akupherezetsa mawuwa osati pongolota kuti adzakhala koma kutsogolera kumene chifukwa nthawi yawo yakwana dailes banda adacheza ndi mwangonde khansala wachinyama yemwe akuimira jumbo ward mumzinda wa mzuzu motere  chisale watuluka nkumangidwanso  sipakala waimitsa nyumba ya malamulo  pa wenela pasintha zedi ali ndi masomphenya mwangonde tikudziweni  ndine lusubilo mwangonde ndili ndi zaka 27 zakubadwa ndinabadwa mbanja la ana asanu ndipo ndine wachinayi kubadwa ndimachokera mmudzi mwa mwamalopa kwa paramount chief kyungu mboma la karonga sindili pabanja pakadalipano mbiri ya maphunziro anu ndi yotani maphunziro anga a pulaimale ndidachitira kusukula yapulaiveti ya viphya mumzinda wa mzuzu ndipo asekondale ndidachitira pa phwezi boys mboma la rumphi ndili ndi diploma ya accounting ndipo pakadalipano

In [11]:
max_fatures = 15000

# Max number of words in each complaint.


#tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer = Tokenizer(num_words=max_fatures, split=' ', filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~0123456789', lower=True)
tokenizer.fit_on_texts(df['post'].values)
X_allw = list(tokenizer.texts_to_sequences(df.post[:].values,))
X_all = list(pad_sequences(X_allw,dtype='float'))
print('Found %s unique tokens.' % len(X_all))

Found 1436 unique tokens.


In [12]:
#pad_lines=list(pad_sequences(lines,dtype='float',value=0.0,padding="post"))

In [13]:
X_all=[list(i) for i in X_all ]

In [14]:
type(X_all)

list

In [15]:
xtestp=0.80
divi=int(len(X_all)*xtestp)
X=X_all[:divi]
X_test=X_all[divi:]
yall=list(df.label_index)
y=yall[:divi]
y_test=yall[divi:]

In [16]:
len(X[0])

828

In [17]:
type(X[0])

list

In [18]:
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta,Adagrad,Nadam
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld

from keras.preprocessing.sequence import pad_sequences

In [19]:
fname="_model_1_"

In [32]:
clear_session()



input_size=max_fatures
input_len=len(X[0])
print(input_len)
output_size=20
drop_frac0=0.6 
drop_frac1=0.6



input1=Input(shape=(input_len,),)
embed=Embedding(input_size,14,input_length=input_len,mask_zero=True,trainable=True)(input1)
lstm1=LSTM(input_len,dropout=0.61,return_sequences=False,return_state=False, go_backwards=True)(embed)
#flatt=Flatten()(lstm1)
d1=Dense(520,activation="relu")(lstm1)
drop0=Dropout(drop_frac0)(d1)

d2=Dense(80,activation="relu")(drop0)
drop1=Dropout(drop_frac1)(d2)



pred=Dense(output_size,activation="softmax")(drop1)

model = Model(inputs=input1, outputs=pred)

opt = tf.keras.optimizers.Adam(learning_rate=0.003)



model.compile(loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=["accuracy"])

828


In [33]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 828)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 828, 14)           210000    
_________________________________________________________________
lstm (LSTM)                  (None, 828)               2792016   
_________________________________________________________________
dense (Dense)                (None, 520)               431080    
_________________________________________________________________
dropout (Dropout)            (None, 520)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 80)                41680     
_________________________________________________________________
dropout_1 (Dropout)          (None, 80)                0     

In [34]:
callbacks = [#callback_LR,
       
        ModelCheckpoint(filepath=fname+"_{loss:.5f}_{val_accuracy:.5f}_.hdf5", monitor='val_accuracy',
                        verbose=2, save_best_only=True, mode='max')]

In [ ]:
history = model.fit( X,y,validation_data=(X_test,y_test), 
                        epochs=150, 
                        batch_size=132,
                        
                        verbose=1,
                        callbacks=callbacks,
                              
                              )

Epoch 1/150
9/9 [==============================] - 20s 2s/step - loss: 3.8528 - accuracy: 0.1186 - val_loss: 2.8727 - val_accuracy: 0.1771

Epoch 00001: val_accuracy improved from -inf to 0.17708, saving model to _model_1__4.05652_0.17708_.hdf5
Epoch 2/150
9/9 [==============================] - 15s 2s/step - loss: 3.2316 - accuracy: 0.0952 - val_loss: 2.8600 - val_accuracy: 0.1771

Epoch 00002: val_accuracy did not improve from 0.17708
Epoch 3/150
9/9 [==============================] - 15s 2s/step - loss: 2.9360 - accuracy: 0.1101 - val_loss: 2.7915 - val_accuracy: 0.1007

Epoch 00003: val_accuracy did not improve from 0.17708
Epoch 4/150
9/9 [==============================] - 15s 2s/step - loss: 2.8298 - accuracy: 0.1169 - val_loss: 2.6947 - val_accuracy: 0.1771

Epoch 00004: val_accuracy did not improve from 0.17708
Epoch 5/150
9/9 [==============================] - 15s 2s/step - loss: 2.8690 - accuracy: 0.1533 - val_loss: 2.7059 - val_accuracy: 0.1771

Epoch 00005: val_accuracy did 